## Pyspark 연습 6

- pandas Dataframe -> pyspark Dataframe 으로 변환
- pyspark ml  로 linear regression model 만들어보기

In [4]:
from pyspark.sql import SparkSession
from sklearn.datasets import load_iris
import pandas as pd
spark = SparkSession.builder.appName('Missing').getOrCreate()

### pandas dataframe 을 pyspark 변환하기

In [2]:
iris = load_iris()

In [5]:
iris_data = iris.data
iris_label = iris.target
iris_df = pd.DataFrame(data = iris_data, columns = iris.feature_names)
iris_df['label'] = iris.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


pandas로 생성한 dataframe은  createDataframe 으로 간단하게 변환할 수 있다.

이걸 몰라서 csv 로 저장하고 불러왔었다..

In [6]:
df = spark.createDataFrame(iris_df)
df.show()

+-----------------+----------------+-----------------+----------------+-----+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|label|
+-----------------+----------------+-----------------+----------------+-----+
|              5.1|             3.5|              1.4|             0.2|    0|
|              4.9|             3.0|              1.4|             0.2|    0|
|              4.7|             3.2|              1.3|             0.2|    0|
|              4.6|             3.1|              1.5|             0.2|    0|
|              5.0|             3.6|              1.4|             0.2|    0|
|              5.4|             3.9|              1.7|             0.4|    0|
|              4.6|             3.4|              1.4|             0.3|    0|
|              5.0|             3.4|              1.5|             0.2|    0|
|              4.4|             2.9|              1.4|             0.2|    0|
|              4.9|             3.1|              1.5|          

In [7]:
df.printSchema()

root
 |-- sepal length (cm): double (nullable = true)
 |-- sepal width (cm): double (nullable = true)
 |-- petal length (cm): double (nullable = true)
 |-- petal width (cm): double (nullable = true)
 |-- label: long (nullable = true)



Vector Assembler

In [16]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)']
                                   ,outputCol= "Independent Features")

In [17]:
output = featureassembler.transform(df)
output.show()

+-----------------+----------------+-----------------+----------------+-----+--------------------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|label|Independent Features|
+-----------------+----------------+-----------------+----------------+-----+--------------------+
|              5.1|             3.5|              1.4|             0.2|    0|       [5.1,3.5,1.4]|
|              4.9|             3.0|              1.4|             0.2|    0|       [4.9,3.0,1.4]|
|              4.7|             3.2|              1.3|             0.2|    0|       [4.7,3.2,1.3]|
|              4.6|             3.1|              1.5|             0.2|    0|       [4.6,3.1,1.5]|
|              5.0|             3.6|              1.4|             0.2|    0|       [5.0,3.6,1.4]|
|              5.4|             3.9|              1.7|             0.4|    0|       [5.4,3.9,1.7]|
|              4.6|             3.4|              1.4|             0.3|    0|       [4.6,3.4,1.4]|
|         

In [18]:
output.columns

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)',
 'label',
 'Independent Features']

vector assembler 에서 생성한 independent feature 값과 예측할 타겟 "petal width (cm)" 만 재선택한다.

In [19]:
finalized_data = output.select("Independent Features", "petal width (cm)")
finalized_data.show()

+--------------------+----------------+
|Independent Features|petal width (cm)|
+--------------------+----------------+
|       [5.1,3.5,1.4]|             0.2|
|       [4.9,3.0,1.4]|             0.2|
|       [4.7,3.2,1.3]|             0.2|
|       [4.6,3.1,1.5]|             0.2|
|       [5.0,3.6,1.4]|             0.2|
|       [5.4,3.9,1.7]|             0.4|
|       [4.6,3.4,1.4]|             0.3|
|       [5.0,3.4,1.5]|             0.2|
|       [4.4,2.9,1.4]|             0.2|
|       [4.9,3.1,1.5]|             0.1|
|       [5.4,3.7,1.5]|             0.2|
|       [4.8,3.4,1.6]|             0.2|
|       [4.8,3.0,1.4]|             0.1|
|       [4.3,3.0,1.1]|             0.1|
|       [5.8,4.0,1.2]|             0.2|
|       [5.7,4.4,1.5]|             0.4|
|       [5.4,3.9,1.3]|             0.4|
|       [5.1,3.5,1.4]|             0.3|
|       [5.7,3.8,1.7]|             0.3|
|       [5.1,3.8,1.5]|             0.3|
+--------------------+----------------+
only showing top 20 rows



In [21]:
from pyspark.ml.regression import LinearRegression
train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol = 'Independent Features', labelCol= 'petal width (cm)')
regressor = regressor.fit(train_data)

22/04/24 21:05:01 WARN Instrumentation: [b8dc8861] regParam is zero, which might cause numerical instability and overfitting.


Coefficients 구하기

In [22]:
regressor.coefficients

DenseVector([-0.1747, 0.1771, 0.5046])

Intercept 구하기

In [23]:
regressor.intercept

-0.21580741349282237

Prediction 하기

In [28]:
pred_results=regressor.evaluate(test_data)
pred_results.predictions.show()

/Users/mijeongkim/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+----------------+--------------------+
|Independent Features|petal width (cm)|          prediction|
+--------------------+----------------+--------------------+
|       [4.8,3.4,1.6]|             0.2|  0.3548647076629381|
|       [4.9,3.0,1.4]|             0.2| 0.16564322904493148|
|       [4.9,3.1,1.5]|             0.1| 0.23380792284680998|
|       [4.9,3.1,1.5]|             0.2| 0.23380792284680998|
|       [5.0,3.4,1.6]|             0.4|  0.3199176270031998|
|       [5.4,3.4,1.7]|             0.2|  0.3004788841434113|
|       [5.5,4.2,1.4]|             0.2| 0.27331329117199976|
|       [5.7,3.8,1.7]|             0.3| 0.31889536452256495|
|       [5.7,4.4,1.5]|             0.4| 0.32424017965633034|
|       [4.4,3.0,1.3]|             0.2| 0.20255551223458904|
|       [4.5,2.3,1.3]|             0.3|0.061117044509388235|
|       [4.6,3.2,1.4]|             0.2|  0.2534824007189195|
|       [4.8,3.0,1.4]|             0.3| 0.18311676937480073|
|       [5.6,2.5,3.9]|  

In [29]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(0.12540277734678246, 0.025090647101851306)